In [ ]:
import pandas as pd
json_file_path = '../data/dataset_autos_V2.jsonl'

df = pd.read_json(json_file_path, lines=True)

In [ ]:
df['id'] = df['id'].astype(str)
df['slug'] = df['slug'].astype(str)
df['sucursal'] = df['sucursal'].astype("category")
df['caja'] = df['caja'].astype("category")
df['details'] = df['details'].astype("string")
df['year'] = df['year'].astype("Int64")
df['km'] = df['km'].astype("Int64")

In [ ]:
df_sorted = df.sort_values('km', na_position='last')
df_final = df_sorted.drop_duplicates(subset=['id'], keep='first')
df_final.to_json('../data/dataset_autos_V2_cleaned.jsonl', orient='records', lines=True)